# Testing WooCommerce Client Integration

Este notebook prueba la integración con WooCommerce REST API:
- Client instantiation
- HTTP Basic Auth
- Order operations (get, mark as paid, update status)
- Error handling

## Setup - Import modules

In [1]:
# Configurar sys.path para poder importar desde app
import sys
from pathlib import Path

# Agregar el directorio backend al path
backend_path = Path.cwd()
if backend_path.name == "backend":
    sys.path.insert(0, str(backend_path))
else:
    # Si estamos en el root del monorepo, navegar a backend
    backend_path = backend_path / "apps" / "backend"
    sys.path.insert(0, str(backend_path))

print(f"Backend path added: {backend_path}")

# Importar los modulos
import asyncio
import json

from app.integrations.woocommerce_client import (
    WooCommerceClient,
    WooCommerceError,
    WooCommerceAuthError,
    WooCommerceNotFoundError,
)

print("Modules imported successfully")

Backend path added: c:\Users\Renzo\Desktop\Proyectos\Ventia\ventia-monorepo\apps\backend
Modules imported successfully


## Test 1: Client Instantiation

Probamos la creacion del cliente WooCommerce con credenciales.

In [ ]:
# Credenciales de prueba (reemplazar con credenciales reales para tests de integracion)
TEST_STORE_URL = "https://example.com"  # Reemplazar con URL real
TEST_CONSUMER_KEY = "ck_xxxx"  # Reemplazar con key real
TEST_CONSUMER_SECRET = "cs_xxxx"  # Reemplazar con secret real

# Crear cliente
client = WooCommerceClient(
    store_url=TEST_STORE_URL,
    consumer_key=TEST_CONSUMER_KEY,
    consumer_secret=TEST_CONSUMER_SECRET,
)

print("WooCommerce Client created successfully")
print(f"  Store URL: {client.store_url}")
print(f"  Base URL: {client.base_url}")
print(f"  Timeout: {client.timeout}s")
print(f"  Auth configured: {client._auth is not None}")

WooCommerce Client created successfully
  Store URL: https://ventia-latam.com
  Base URL: https://ventia-latam.com/wp-json/wc/v3
  Timeout: 30.0s
  Auth configured: True


## Test 2: URL Normalization

Verificamos que el cliente normalice correctamente las URLs.

In [ ]:
# Test con trailing slash
client_with_slash = WooCommerceClient(
    store_url="https://example.com/",
    consumer_key="ck_test",
    consumer_secret="cs_test",
)

print("URL Normalization Tests:")
print(f"  Input: 'https://example.com/'")
print(f"  Store URL: {client_with_slash.store_url}")
print(f"  Base URL: {client_with_slash.base_url}")

# Verificar que no tenga doble slash
assert client_with_slash.store_url == "https://example.com"
assert client_with_slash.base_url == "https://example.com/wp-json/wc/v3"
print("\nURL normalization working correctly")

## Test 3: Exception Classes

Verificamos que las excepciones personalizadas funcionen correctamente.

In [3]:
print("Exception Classes Tests:\n")

# Test WooCommerceError
try:
    raise WooCommerceError("Test error message", status_code=500)
except WooCommerceError as e:
    print(f"WooCommerceError:")
    print(f"  Message: {e.message}")
    print(f"  Status Code: {e.status_code}")
    print(f"  str(e): {str(e)}")

# Test WooCommerceAuthError
try:
    raise WooCommerceAuthError("Invalid credentials", status_code=401)
except WooCommerceAuthError as e:
    print(f"\nWooCommerceAuthError:")
    print(f"  Message: {e.message}")
    print(f"  Status Code: {e.status_code}")
    print(f"  Is subclass of WooCommerceError: {isinstance(e, WooCommerceError)}")

# Test WooCommerceNotFoundError
try:
    raise WooCommerceNotFoundError("Order not found", status_code=404)
except WooCommerceNotFoundError as e:
    print(f"\nWooCommerceNotFoundError:")
    print(f"  Message: {e.message}")
    print(f"  Status Code: {e.status_code}")
    print(f"  Is subclass of WooCommerceError: {isinstance(e, WooCommerceError)}")

print("\nAll exception classes working correctly")

Exception Classes Tests:

WooCommerceError:
  Message: Test error message
  Status Code: 500
  str(e): Test error message

WooCommerceAuthError:
  Message: Invalid credentials
  Status Code: 401
  Is subclass of WooCommerceError: True

WooCommerceNotFoundError:
  Message: Order not found
  Status Code: 404
  Is subclass of WooCommerceError: True

All exception classes working correctly


## Test 4: Client Methods Exist

Verificamos que todos los metodos requeridos existan en el cliente.

In [4]:
print("Client Methods Verification:\n")

required_methods = [
    "_request",
    "get_order",
    "mark_order_as_paid",
    "update_order_status",
    "create_order",
]

for method_name in required_methods:
    has_method = hasattr(client, method_name)
    is_callable = callable(getattr(client, method_name, None))
    status = "OK" if has_method and is_callable else "MISSING"
    print(f"  {status} {method_name}()")

# Verificar que sean async
print("\nAsync Method Verification:")
import inspect
for method_name in required_methods:
    method = getattr(client, method_name)
    is_async = inspect.iscoroutinefunction(method)
    status = "async" if is_async else "sync"
    print(f"  {method_name}(): {status}")

print("\nAll required methods exist and are async")

Client Methods Verification:

  OK _request()
  OK get_order()
  OK mark_order_as_paid()
  OK update_order_status()
  OK create_order()

Async Method Verification:
  _request(): async
  get_order(): async
  mark_order_as_paid(): async
  update_order_status(): async
  create_order(): async

All required methods exist and are async


## Test 5: Integration with Tenant Settings

Probamos que el cliente se pueda crear desde las credenciales del tenant.

In [ ]:
from app.schemas.tenant_settings import (
    WooCommerceCredentials,
    EcommerceSettings,
    TenantSettings,
)

# Simular credenciales de un tenant
woo_creds = WooCommerceCredentials(
    store_url="https://mi-tienda.com",
    consumer_key="ck_live_key_123",
    consumer_secret="cs_live_secret_456",
)

ecommerce_settings = EcommerceSettings(
    sync_on_validation=True,
    woocommerce=woo_creds,
)

tenant_settings = TenantSettings(ecommerce=ecommerce_settings)

print("Tenant Settings Integration Test:\n")
print(f"  Platform: {tenant_settings.platform}")
print(f"  Has E-commerce: {tenant_settings.has_ecommerce}")
print(f"  Sync on validation: {tenant_settings.ecommerce.sync_on_validation}")

# Crear cliente desde credenciales del tenant
if tenant_settings.platform == "woocommerce":
    creds = tenant_settings.ecommerce.woocommerce
    client_from_tenant = WooCommerceClient(
        store_url=creds.store_url,
        consumer_key=creds.consumer_key,
        consumer_secret=creds.consumer_secret,
    )
    print(f"\nClient created from tenant settings:")
    print(f"  Store URL: {client_from_tenant.store_url}")
    print(f"  Base URL: {client_from_tenant.base_url}")

print("\nIntegration with tenant settings working correctly")

## Test 6: Real API Calls (OPTIONAL)

**ADVERTENCIA**: Este test hace llamadas REALES a la API de WooCommerce.
Solo ejecutar si tienes credenciales validas y una tienda de prueba.

Configura las credenciales reales en las variables de abajo.

In [ ]:
# === CONFIGURAR CREDENCIALES REALES AQUI ===
REAL_STORE_URL = "https://mi-tienda.com"  # ej: "https://mi-tienda.com"
REAL_CONSUMER_KEY = "ck_xxxx"  # ej: "ck_xxxx"
REAL_CONSUMER_SECRET = "cs_xxxx"  # ej: "cs_xxxx"
TEST_ORDER_ID = None  # ej: 123 (ID de una orden existente)

# Verificar si hay credenciales configuradas
if REAL_STORE_URL and REAL_CONSUMER_KEY and REAL_CONSUMER_SECRET:
    print("Credenciales configuradas")
    print(f"  Store: {REAL_STORE_URL}")
    print(f"  Consumer Key: {REAL_CONSUMER_KEY[:10]}...")
else:
    print("No hay credenciales configuradas.")
    print("Configura REAL_STORE_URL, REAL_CONSUMER_KEY y REAL_CONSUMER_SECRET para probar.")

Credenciales configuradas
  Store: https://anamarialajusticiaperu.pe
  Consumer Key: ck_f8b7ff8...


In [9]:
# Test: Get Order (solo si hay credenciales y order ID)
async def test_get_order():
    if not (REAL_STORE_URL and REAL_CONSUMER_KEY and REAL_CONSUMER_SECRET and TEST_ORDER_ID):
        print("Skipping: No credentials or order ID configured")
        return
    
    client = WooCommerceClient(
        store_url=REAL_STORE_URL,
        consumer_key=REAL_CONSUMER_KEY,
        consumer_secret=REAL_CONSUMER_SECRET,
    )
    
    print(f"Fetching order {TEST_ORDER_ID}...\n")
    
    try:
        order = await client.get_order(TEST_ORDER_ID)
        print("Order retrieved successfully!")
        print(f"  ID: {order.get('id')}")
        print(f"  Number: {order.get('number')}")
        print(f"  Status: {order.get('status')}")
        print(f"  Total: {order.get('total')} {order.get('currency')}")
        print(f"  Customer: {order.get('billing', {}).get('email')}")
        print(f"  Date Created: {order.get('date_created')}")
        print(f"  Date Paid: {order.get('date_paid')}")
        print(f"  Payment Method: {order.get('payment_method_title')}")
        print(f"  Line Items: {len(order.get('line_items', []))}")
        
        # Mostrar items
        for item in order.get('line_items', []):
            print(f"    - {item.get('name')} x{item.get('quantity')} = {item.get('total')}")
            
    except WooCommerceNotFoundError as e:
        print(f"Order not found: {e}")
    except WooCommerceAuthError as e:
        print(f"Authentication failed: {e}")
    except WooCommerceError as e:
        print(f"API Error: {e}")

# Ejecutar test
await test_get_order()

Fetching order 8058...

Order retrieved successfully!
  ID: 8058
  Number: 8058
  Status: completed
  Total: 602.90 PEN
  Customer: rut.stamaria@gmail.com
  Date Created: 2026-01-17T12:40:14
  Date Paid: 2026-01-17T12:43:13
  Payment Method: Pago con tarjeta de crédito
  Line Items: 2
    - COLAGENO CON MAGNESIO + VIT.C + VIT.B1, B2, B6 (sabor fresa) - polvo x2 = 286.44
    - LECITINA DE SOJA x1 = 203.31


In [4]:
# Test: List Recent Orders (solo si hay credenciales)
async def test_list_orders():
    if not (REAL_STORE_URL and REAL_CONSUMER_KEY and REAL_CONSUMER_SECRET):
        print("Skipping: No credentials configured")
        return
    
    client = WooCommerceClient(
        store_url=REAL_STORE_URL,
        consumer_key=REAL_CONSUMER_KEY,
        consumer_secret=REAL_CONSUMER_SECRET,
    )
    
    print("Fetching recent orders...\n")
    
    try:
        # Usar _request directamente para listar ordenes
        orders = await client._request("GET", "/orders?per_page=5")
        
        print(f"Found {len(orders)} orders:\n")
        for order in orders:
            print(f"  Order #{order.get('number')} (ID: {order.get('id')})")
            print(f"    Status: {order.get('status')}")
            print(f"    Total: {order.get('total')} {order.get('currency')}")
            print(f"    Customer: {order.get('billing', {}).get('email')}")
            print()
            
    except WooCommerceAuthError as e:
        print(f"Authentication failed: {e}")
    except WooCommerceError as e:
        print(f"API Error: {e}")

# Ejecutar test
await test_list_orders()

Fetching recent orders...

Found 0 orders:



In [ ]:
# Test: Mark Order as Paid (CUIDADO - modifica datos reales)
# Descomentar solo si quieres marcar una orden como pagada

async def test_mark_as_paid():
    if not (REAL_STORE_URL and REAL_CONSUMER_KEY and REAL_CONSUMER_SECRET and TEST_ORDER_ID):
        print("Skipping: No credentials or order ID configured")
        return
    
    client = WooCommerceClient(
        store_url=REAL_STORE_URL,
        consumer_key=REAL_CONSUMER_KEY,
        consumer_secret=REAL_CONSUMER_SECRET,
    )
    
    print(f"Marking order {TEST_ORDER_ID} as paid...\n")
    print("WARNING: This will modify the order in WooCommerce!\n")
    
    try:
        updated_order = await client.mark_order_as_paid(TEST_ORDER_ID)
        
        print("Order marked as paid successfully!")
        print(f"  ID: {updated_order.get('id')}")
        print(f"  Status: {updated_order.get('status')}")
        print(f"  Date Paid: {updated_order.get('date_paid')}")
        
    except WooCommerceNotFoundError as e:
        print(f"Order not found: {e}")
    except WooCommerceAuthError as e:
        print(f"Authentication failed: {e}")
    except WooCommerceError as e:
        print(f"API Error: {e}")

# DESCOMENTAR PARA EJECUTAR (CUIDADO!):
# await test_mark_as_paid()
print("Test mark_order_as_paid commented out for safety.")
print("Uncomment the last line to run.")

In [ ]:
# Test: Update Order Status (CUIDADO - modifica datos reales)
# Descomentar solo si quieres cambiar el estado de una orden

async def test_update_status():
    if not (REAL_STORE_URL and REAL_CONSUMER_KEY and REAL_CONSUMER_SECRET and TEST_ORDER_ID):
        print("Skipping: No credentials or order ID configured")
        return
    
    client = WooCommerceClient(
        store_url=REAL_STORE_URL,
        consumer_key=REAL_CONSUMER_KEY,
        consumer_secret=REAL_CONSUMER_SECRET,
    )
    
    NEW_STATUS = "completed"  # pending, processing, on-hold, completed, cancelled, refunded, failed
    
    print(f"Updating order {TEST_ORDER_ID} status to '{NEW_STATUS}'...\n")
    print("WARNING: This will modify the order in WooCommerce!\n")
    
    try:
        updated_order = await client.update_order_status(TEST_ORDER_ID, NEW_STATUS)
        
        print("Order status updated successfully!")
        print(f"  ID: {updated_order.get('id')}")
        print(f"  New Status: {updated_order.get('status')}")
        
    except WooCommerceNotFoundError as e:
        print(f"Order not found: {e}")
    except WooCommerceAuthError as e:
        print(f"Authentication failed: {e}")
    except WooCommerceError as e:
        print(f"API Error: {e}")

# DESCOMENTAR PARA EJECUTAR (CUIDADO!):
# await test_update_status()
print("Test update_order_status commented out for safety.")
print("Uncomment the last line to run.")

## Test 7: Error Handling Simulation

Simulamos diferentes escenarios de error para verificar el manejo.

In [ ]:
import httpx

print("Error Handling Simulation:\n")

# Test con URL invalida (deberia fallar en conexion)
async def test_connection_error():
    client = WooCommerceClient(
        store_url="https://invalid-store-that-does-not-exist-12345.com",
        consumer_key="ck_test",
        consumer_secret="cs_test",
        timeout=5.0,  # Timeout corto para no esperar mucho
    )
    
    try:
        await client.get_order(1)
        print("ERROR: Should have raised an exception")
    except httpx.RequestError as e:
        print(f"Connection Error (expected): {type(e).__name__}")
    except Exception as e:
        print(f"Other Error: {type(e).__name__}: {e}")

await test_connection_error()
print("\nError handling working correctly")

## Resumen de Tests

Este notebook ha probado:

**Client Instantiation** - Creacion del cliente con credenciales  
**URL Normalization** - Normalizacion de URLs (trailing slashes)  
**Exception Classes** - Excepciones personalizadas funcionan correctamente  
**Client Methods** - Todos los metodos requeridos existen y son async  
**Tenant Integration** - Integracion con TenantSettings funciona  
**Error Handling** - Manejo de errores de conexion  

### WooCommerce REST API v3

El cliente implementa:

**Autenticacion:**
- HTTP Basic Auth con consumer_key:consumer_secret
- Base URL: `{store_url}/wp-json/wc/v3`

**Metodos disponibles:**
- `get_order(order_id)` - Obtener detalles de una orden
- `mark_order_as_paid(order_id)` - Marcar orden como pagada (set_paid=true)
- `update_order_status(order_id, status)` - Actualizar estado de orden
- `create_order(order_data)` - Crear nueva orden

**Estados de orden validos:**
- `pending` - Pendiente de pago
- `processing` - En proceso (pagada)
- `on-hold` - En espera
- `completed` - Completada
- `cancelled` - Cancelada
- `refunded` - Reembolsada
- `failed` - Fallida

**LISTO PARA INTEGRACION** ✓